In [8]:
pip install pyhwpx


Note: you may need to restart the kernel to use updated packages.


In [7]:
-m pip install --upgrade pip

SyntaxError: invalid syntax (2350214200.py, line 1)

In [ ]:
from pyhwpx import Hwp
import time
# Hwp 인스턴스 생성
hwp = Hwp()
hwp.Run("FileOpen") #파일 오픈
Table_list = [] # 표 인덱스 저장
for i in hwp.ctrl_list:
    if i.UserDesc == "표":#컨트롤이 표일 경우
        Table_list.append(i)#리스트에 저장
print(Table_list)

for i in Table_list:    
    hwp.move_to_ctrl(i) #표 앞으로 커서 이동
    hwp.find_ctrl() #표 선택
    hwp.ShapeObjTableSelCell() #표 첫번째 셀 선택
    hwp.TableCellBlock() #3번하면 올블록됨
    hwp.TableCellBlockExtend()
    hwp.TableCellBlockExtend()
    time.sleep(2)


In [4]:
from pyhwpx import Hwp
import time
# Hwp 인스턴스 생성
hwp = Hwp()
hwp.Run("FileOpen") #파일 오픈

Table_list = [] # 모든 표의 갯수 찾기
for i in hwp.ctrl_list:
    if i.UserDesc == "표":#컨트롤이 표일 경우
        Table_list.append(i)#리스트에 저장
Table_Total = len(Table_list) # 문서안의 모든 표의 갯수
Table_index = 2 #표 인덱스
print(Table_Total)
print(Table_index)
Table_index = max(0, min(Table_index+1, Table_Total))
print(Table_index)

def 표라인_전체투명():
    pset = hwp.HParameterSet.HCellBorderFill
    hwp.HAction.GetDefault("CellBorder", pset.HSet)
    pset.TypeHorz = hwp.HwpLineType("None")
    pset.BorderTypeBottom = hwp.HwpLineType("None")
    pset.BorderTypeTop = hwp.HwpLineType("None")
    pset.BorderTypeRight = hwp.HwpLineType("None")
    pset.BorderTypeLeft = hwp.HwpLineType("None")
    hwp.HAction.Execute("CellBorder", pset.HSet)
def 표라인_안쪽실선():
    pset = hwp.HParameterSet.HCellBorderFill
    hwp.HAction.GetDefault("CellBorder", pset.HSet)
    pset.WidthVert = hwp.HwpLineWidth("0.12mm")
    pset.TypeVert = hwp.HwpLineType("Solid")
    pset.TypeHorz = hwp.HwpLineType("Solid")
    hwp.HAction.Execute("CellBorder", pset.HSet)
def 표라인_위아래굵은선():
    pset = hwp.HParameterSet.HCellBorderFill
    hwp.HAction.GetDefault("CellBorder", pset.HSet)
    pset.BorderWidthBottom = hwp.HwpLineWidth("0.4mm")
    pset.BorderTypeBottom = hwp.HwpLineType("Solid")
    pset.BorderWidthTop = hwp.HwpLineWidth("0.4mm")
    pset.BorderTypeTop = hwp.HwpLineType("Solid")
    hwp.HAction.Execute("CellBorder", pset.HSet)
def 표색_없음():
    pset = hwp.HParameterSet.HCellBorderFill
    hwp.HAction.GetDefault("CellBorder", pset.HSet)
    pset.FillAttr.Type = hwp.BrushType("NullBrush")
    pset.FillAttr.GradationAlpha = 0
    pset.FillAttr.WindowsBrush = 0
    pset.FillAttr.ImageAlpha = 0
    hwp.HAction.Execute("CellBorder", pset.HSet)

hwp.get_into_nth_table(Table_index)#인덱스 표의 첫번째 셀로 이동
hwp.TableCellBlock() #3번하면 올블록됨
hwp.TableCellBlockExtend()
hwp.TableCellBlockExtend()
표라인_전체투명()
표라인_안쪽실선()
표라인_위아래굵은선()
#표색_없음()

7
2
3


In [ ]:
from pyhwpx import Hwp
import time
# Hwp 인스턴스 생성
hwp = Hwp()
hwp.clipboard_to_pyfunc()

In [5]:
import tkinter as tk
from tkinter import ttk
from pyhwpx import Hwp

#region인스턴스 생성
hwp = Hwp()

# Tkinter 창 생성
root = tk.Tk()
root.title("표 셀배경 / 선 매크로")
#endregion

#region 파일 경로 표시
file_path_frame = tk.Frame(root, highlightbackground="black", highlightthickness=1, padx=10, pady=5)
file_path_frame.pack(fill="x", padx=10, pady=5)

file_path_label = tk.Label(file_path_frame, text="파일 경로: 없음", font=("Arial", 10), anchor="w")
file_path_label.pack(side="left", fill="x", expand=True, padx=5)

hwp.Run("FileOpen")
def update_file_path_label():
    """파일 경로를 업데이트하고 Label에 표시"""
    if hwp.Path:
        file_path = hwp.Path
        file_path_label.config(text=f"파일 경로: {file_path}")
        print(f"파일 경로: {file_path}")
    else:
        file_path_label.config(text="파일 경로: 없음")
        print("파일 경로: 없음")

def select_file():
    """파일을 재선택"""
    try:
        hwp.Save()
        hwp.Run("FileClose")
        hwp.Run("FileOpen")
        update_file_path_label()
    except Exception as e:
        print(f"파일 재선택 중 오류: {e}")

file_reselect_button = tk.Button(file_path_frame, text="파일 재선택", command=select_file)
file_reselect_button.pack(side="right", padx=5)

#표 초기화
Table_list = [] # 모든 표의 갯수 찾기
for i in hwp.ctrl_list:
    if i.UserDesc == "표":#컨트롤이 표일 경우
        Table_list.append(i)#리스트에 저장
Table_Total = len(Table_list) # 문서안의 모든 표의 갯수
print(f"전체 표 갯수 : {Table_Total}")
Table_index = 0 #표 인덱스

# Notebook(탭 컨트롤러) 생성
notebook = ttk.Notebook(root)
notebook.pack(expand=True, fill="both", padx=10, pady=10)
#endregion

# region 함수 모음
def on_CellLineMacro(event=None):
    """버튼 클릭 이벤트: 드롭다운 값 출력"""
    first_option = dropdown1.get()
    second_option = dropdown2.get()
    print(f"첫행 배경색: {first_option}, 첫행 밑 줄: {second_option}")
    #여기에 자동화 함수 추가

def 그림용(event=None):
    print("그림용")
    """그림 표용 세팅"""

def on_CommonlocationMacro(event=None):
    """공통 표 위치 매크로"""
    print("공통매크로")
    #여기 표위치 공통적으로 지정할 것 추가

def handle_1단용(event=None):
    print("1단용 버튼 클릭!")
    on_CommonlocationMacro()
    # 1단용 버튼 동작
def handle_2단_왼_상(event=None):
    print("2단 왼 상 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단 왼 상 동작
def handle_2단_가운_상(event=None):
    print("2단 가운 상 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단 가운 상 동작
def handle_2단_오른_상(event=None):
    print("2단 오른 상 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단 오른 상 동작

def handle_2단_왼_하(event=None):
    print("2단_왼_하 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단_왼_하 동작
def handle_2단_가운_하(event=None):
    print("2단_가운_하 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단_가운_하 동작
def handle_2단_오른_하(event=None):
    print("2단_오른_하 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단_오른_하 동작

def 이전표(event=None):
    global Table_index
    # Spinbox에서 현재 값을 가져오고 -1
    current_value = current_table_index.get()
    prev_index = max(0, current_value - 1)  # 최소값 제한
    current_table_index.set(prev_index)  # Spinbox에 값 설정
    Table_index = prev_index  # 전역 변수 업데이트
    print(f"현재 Table_index: {Table_index}")
    hwp.get_into_nth_table(Table_index)#인덱스 표의 첫번째 셀로 이동

def 다음표(event=None):
    global Table_index
    # Spinbox에서 현재 값을 가져오고 +1
    current_value = current_table_index.get()
    next_index = max(0, min(current_value + 1, Table_Total - 1))  # 최대값 제한
    current_table_index.set(next_index)  # Spinbox에 값 설정
    Table_index = next_index  # 전역 변수 업데이트
    print(f"현재 Table_index: {Table_index}")
    hwp.get_into_nth_table(Table_index)#인덱스 표의 첫번째 셀로 이동

def 양옆맞추기(event=None):
    print("양옆맞추기")
    """양옆맞추기"""
def 처음으로(event=None):
    """처음으로"""
    global Table_index
    # Spinbox와 Table_index를 동기화
    Table_index = 0  # 전역 변수 Table_index를 0으로 설정
    current_table_index.set(Table_index)  # Spinbox에 값 반영
    try:
        hwp.get_into_nth_table(Table_index)  # 첫 번째 표로 이동
        print(f"처음으로: Table_index = {Table_index}")
    except Exception as e:
        print(f"처음으로 이동 중 오류 발생: {e}")

#endregion

#region 첫번째 탭 : 표 셀/선 매크로
##########################################
tab1 = ttk.Frame(notebook)
notebook.add(tab1, text="표 셀/선 매크로")

# '표 이동' 프레임
frame_table_movement = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame_table_movement.grid(row=0, column=0, padx=30, pady=10)

position_description = tk.Label(frame_table_movement, text="표 이동", font=("Arial", 12, "bold"))
position_description.grid(row=0, column=0, columnspan=4, padx=5, pady=5)

current_table_index = tk.IntVar(value=0)

# 현재 표 인덱스와 이동 버튼
current_index_label = tk.Label(frame_table_movement, text="현재 표 인덱스:")
current_index_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")

index_spinbox = tk.Spinbox(frame_table_movement, from_=0, to=0, textvariable=current_table_index, width=5)
index_spinbox.grid(row=1, column=1, padx=5, pady=5)

btn_prev = tk.Button(frame_table_movement, text="이전", command=이전표)
btn_prev.grid(row=2, column=0, padx=5, pady=5)

btn_next = tk.Button(frame_table_movement, text="다음", command=다음표)
btn_next.grid(row=2, column=1, padx=5, pady=5)

btn_align = tk.Button(frame_table_movement, text="양옆맞추기", command=양옆맞추기)
btn_align.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

btn_begin = tk.Button(frame_table_movement, text="처음으로", command=처음으로)
btn_begin.grid(row=3, column=3, columnspan=1, padx=5, pady=5)

# '배경/선 매크로' 프레임
frame_cell_macro = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame_cell_macro.grid(row=1, column=0, padx=30, pady=10)

cell_macro_label = tk.Label(frame_cell_macro, text="배경/선 매크로", font=("Arial", 12, "bold"))
cell_macro_label.grid(row=0, column=0, columnspan=2, padx=5, pady=5)

dropdown1_label = tk.Label(frame_cell_macro, text="첫행 배경색:")
dropdown1_label.grid(row=1, column=0, padx=5, pady=5)

dropdown1 = ttk.Combobox(frame_cell_macro, values=["없음", "회색 255,255,255,15%"])
dropdown1.set("회색 255,255,255,15%")
dropdown1.grid(row=1, column=1, padx=5, pady=5)

dropdown2_label = tk.Label(frame_cell_macro, text="첫행 밑 줄:")
dropdown2_label.grid(row=2, column=0, padx=5, pady=5)

dropdown2 = ttk.Combobox(frame_cell_macro, values=["한줄", "두줄"])
dropdown2.set("한줄")
dropdown2.grid(row=2, column=1, padx=5, pady=5)

btn_execute = tk.Button(frame_cell_macro, text="실행", command=lambda: print(f"첫행 배경색: {dropdown1.get()}, 첫행 밑 줄: {dropdown2.get()}"))
btn_execute.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

# '표 위치 매크로' 프레임 추가
frame1 = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame1.grid(row=3, column=0, columnspan=2, padx=30, pady=7)

# 설명 레이블
position_description = tk.Label(frame1, text="표 위치 매크로", font=("Arial", 12, "bold"))
position_description.grid(row=0, column=0, columnspan=4, padx=10, pady=7)

# 오른 단독 버튼
left_button = tk.Button(frame1, text="1단용", command=handle_1단용)
left_button.grid(row=1, column=3, rowspan=2, padx=10, pady=7)


# 6개의 버튼 배열 (1열 3개, 2열 3개)
button1 = tk.Button(frame1, text="2단 왼 상", command=handle_2단_왼_상)
button1.grid(row=1, column=0, padx=10, pady=7)

button2 = tk.Button(frame1, text="2단 가운 상", command=handle_2단_가운_상)
button2.grid(row=1, column=1, padx=10, pady=7)

button3 = tk.Button(frame1, text="2단 오른 상", command=handle_2단_오른_상)
button3.grid(row=1, column=2, padx=10, pady=7)

button4 = tk.Button(frame1, text="2단 왼 하", command=handle_2단_왼_하)
button4.grid(row=2, column=0, padx=10, pady=7)

button5 = tk.Button(frame1, text="2단 가운 하", command=handle_2단_가운_하)
button5.grid(row=2, column=1, padx=10, pady=7)

button6 = tk.Button(frame1, text="2단 오른 하", command=handle_2단_오른_하)
button6.grid(row=2, column=2, padx=10, pady=7)

# '표 위치 매크로' 프레임 추가
frame1 = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame1.grid(row=0, column=2, columnspan=2, padx=30, pady=7)

# 키 이벤트 연결
root.bind("<Control-F1>", on_CellLineMacro)
#endregion

#region 나중에 만들거
########################################################
# 두 번째 탭
tab2 = ttk.Frame(notebook)
notebook.add(tab2, text="추가 기능 1")

frame2 = tk.Frame(tab2, padx=10, pady=10)
frame2.pack(expand=True, fill="both")
label_tab2 = tk.Label(frame2, text="여기에 두 번째 탭의 내용을 추가하세요.", font=("Arial", 12))
label_tab2.pack(pady=20)

# 세 번째 탭
tab3 = ttk.Frame(notebook)
notebook.add(tab3, text="추가 기능 2")

frame3 = tk.Frame(tab3, padx=10, pady=10)
frame3.pack(expand=True, fill="both")
label_tab3 = tk.Label(frame3, text="여기에 세 번째 탭의 내용을 추가하세요.", font=("Arial", 12))
label_tab3.pack(pady=20)
##############################################################
#endregion

# UI 출력
root.update_idletasks()
root.geometry(f"{max(200, notebook.winfo_reqwidth())}x{max(300, notebook.winfo_reqheight() + 100)}")

# 파일 경로 업데이트
update_file_path_label()

# Tkinter 메인 루프 실행
root.mainloop()


전체 표 갯수 : 7
파일 경로: D:\Unity\git\hncAto\ddd.hwp


KeyboardInterrupt: 